In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import json
import time
from tqdm.notebook import tqdm 

<h4>Start cells here if {state}_geo file has not been FIPS encoded

In [4]:
fips_check = pd.read_csv("data/state_data/geo/geo_al.csv") #- original geocoded file
fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head() #10912

,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,Veteran,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long
0,1517597200,4/15/20,459,PPP,ACTION ENTERPRISE HOLDINGS LLC,204 20TH ST N,BIRMINGHAM,AL,35203-3610,8/5/21,...,Non-Veteran,NaN,10124657.53,7/27/21,48.0,Transportation and Warehousing,ALABAMA DISTRICT OFFICE,"204 20TH ST N, BIRMINGHAM, AL",33.515571,-86.806634
1,1798427201,4/15/20,459,PPP,NAPHCARE US INC,2090 Columbiana Road,VESTAVIA HILLS,AL,35216-2153,7/14/21,...,Veteran,NaN,10112876.71,6/11/21,62.0,Health Care and Social Assistance,ALABAMA DISTRICT OFFICE,"2090 Columbiana Road, VESTAVIA HILLS, AL",33.435789,-86.811428
2,2886767410,5/6/20,459,PPP,"AMERICAN FAMILY CARE, INC.",3700 Cahaba Beach Road,BIRMINGHAM,AL,35242,7/17/21,...,Unanswered,NaN,10108611.11,6/11/21,62.0,Health Care and Social Assistance,ALABAMA DISTRICT OFFICE,"3700 Cahaba Beach Road, BIRMINGHAM, AL",33.431916,-86.689670
3,4982277110,4/13/20,459,PPP,"MAYNARD COOPER & GALE, PC.",1901 6TH AVE N Ste 2400,BIRMINGHAM,AL,35203-2602,9/24/21,...,Unanswered,NaN,10132328.77,8/18/21,54.0,"Professional, Scientific, and Technical Services",ALABAMA DISTRICT OFFICE,"1901 6TH AVE N Ste 2400, BIRMINGHAM, AL",33.518619,-86.809388
4,5001327110,4/13/20,459,PPP,"QUALITY RESTAURANT CONCEPTS, LLC",VESTAVIA PKWY,VESTAVIA,AL,35216-1327,7/9/21,...,Non-Veteran,NaN,10114722.22,6/11/21,72.0,Accomodation and Food Services,ALABAMA DISTRICT OFFICE,"VESTAVIA PKWY, VESTAVIA, AL",33.418736,-86.797244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10907,9206768503,3/12/21,459,PPS,PRIESTER RETAIL STORES INC.,1575 Old Fort Rd E,Fort Deposit,AL,36032-4743,9/25/21,...,Non-Veteran,NaN,150604.11,8/11/21,42.0,Wholesale Trade,ALABAMA DISTRICT OFFICE,"1575 Old Fort Rd E, Fort Deposit, AL",31.975720,-86.603696
10908,9298908504,3/12/21,459,PPS,AGS DISTRIBUTION INC.,685 Fox Trl,Opelika,AL,36801-4639,NaN,...,Non-Veteran,NaN,NaN,NaN,33.0,Manufacturing,ALABAMA DISTRICT OFFICE,"685 Fox Trl, Opelika, AL",32.650615,-85.358977
10909,9323947403,5/20/20,459,PPP,ROBERTO REYES DRYWALL INC,221 Wagon Trail,Alabaster,AL,35007-5823,9/22/21,...,Unanswered,NaN,151841.10,8/13/21,23.0,Construction,ALABAMA DISTRICT OFFICE,"221 Wagon Trail, Alabaster, AL",33.256737,-86.828368
10910,9406888509,3/12/21,459,PPS,OAKRIDGE SERVICES LLC,5685B I 10 Industrial Pkwy N,Theodore,AL,36582-1661,9/24/21,...,Non-Veteran,NaN,150427.40,8/27/21,56.0,Administrative and Support and Waste Managemen...,ALABAMA DISTRICT OFFICE,"5685B I 10 Industrial Pkwy N, Theodore, AL",30.568623,-88.175929


In [5]:
columns = ['LoanNumber','full_add', 'Lat', 'Long']
fips_check = fips_check[columns]

In [7]:
fips_check_nan = fips_check[fips_check["Lat"].isnull()] # check to make sure no nulls in coords
fips_check_nan 

,LoanNumber,full_add,Lat,Long
1097,6283588301,"10601 Highway 43, Creola, AL",NaN,NaN
4317,9630887104,"16213 HIGHWAY 171, NORTHPORT, AL",NaN,NaN
5500,2016057209,"801 FRANKLIN DR, TROY, AL",NaN,NaN
6912,6805988301,"8601 Hwy. 227 N., Crossville, AL",NaN,NaN
6914,6666597207,"8601 AL 227, CROSSVILLE, AL",NaN,NaN
8370,9787997710,"2656 Queenstown Road, Birmingham, AL",NaN,NaN
10060,1414547106,"1123 SCHILLINGER RD, MOBILE, AL",NaN,NaN
10685,9414367009,"49265 HIGHWAY 22, WADLEY, AL",NaN,NaN


In [10]:
fips_check_notna = fips_check[fips_check["Lat"].notna()] # check to make sure no nulls in coords
fips_check_notna.head() #10904

,LoanNumber,full_add,Lat,Long
0,1517597200,"204 20TH ST N, BIRMINGHAM, AL",33.515571,-86.806634
1,1798427201,"2090 Columbiana Road, VESTAVIA HILLS, AL",33.435789,-86.811428
2,2886767410,"3700 Cahaba Beach Road, BIRMINGHAM, AL",33.431916,-86.689670
3,4982277110,"1901 6TH AVE N Ste 2400, BIRMINGHAM, AL",33.518619,-86.809388
4,5001327110,"VESTAVIA PKWY, VESTAVIA, AL",33.418736,-86.797244


<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [ ]:
fips_check_scraped = pd.read_csv("data/state_data/geo/geo_fips/AL_fips_scraped.csv") 
fips_check_scraped.head()

In [69]:
columns = ['LoanNumber','full_add', 'Lat', 'Long', 'FIPS_z']
fips_check_scraped = fips_check_scraped[columns]

In [ ]:
#pd.set_option('display.float_format', '{:.0f}'.format)

In [70]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() #15792

,LoanNumber,full_add,Lat,Long,FIPS_z
0,2008847101,"9366 East Raintree Drive, SCOTTSDALE, AZ",33.620398,-111.878704,4.013217e+13
1,3057347105,"PO Box 170, FORT DEFIANCE, AZ",35.744460,-109.076483,4.001944e+13
2,3470877104,"2999 N44th Street Suite 100, Phoenix, AZ",33.482323,-111.986368,4.013111e+13
3,3530847305,"5055 E BROADWAY BLVD Ste A100, TUCSON, AZ",32.222108,-110.884676,4.019003e+13
4,3771367108,"8880 E CHAPPARAL RD SUITE 300, SCOTTSDALE, AZ",33.509368,-111.889853,4.013941e+13


In [73]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null.head() #8655

,LoanNumber,full_add,Lat,Long,FIPS_z
736,3445788509,"PRIDE BAKERIES LLC, PHOENIX, AZ",38.740566,-90.297851,NaN
4066,4410207007,"P.O. Box 19460, PHOENIX, AZ",40.118071,-75.539316,NaN
4098,1641998609,"1762 Highway 95, Bullhead City, AZ",43.844102,-116.953670,NaN
5487,8620287005,"6240 Tacoma Mall Blvd. Ste 315, QUEEN CREEK, AZ",47.199526,-122.463532,NaN
6154,4214117203,"1762 HIGHWAY 95, Bullhead City, AZ",43.844102,-116.953670,NaN


<h4>FIPS Script

In [11]:
#if geocode file
lat = fips_check_notna["Lat"].astype(float).astype(str).to_list()
long = fips_check_notna["Long"].astype(float).astype(str).to_list()

#if FIPS_z file 
#lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
#long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [13]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
    
executable_path = {'executable_path': ChromeDriverManager().install()}
driver = Browser('chrome', **executable_path, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('_x_id').clear()
        driver.find_by_id('_y_id').clear()
        driver.find_by_id('_x_id').fill(long[i])
        driver.find_by_id('_y_id').fill(lat[i])
        driver.find_by_id('proceed').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='pl_gov_census_geo_geocoder_domain_GeographyResult')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('01')] #AL
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                notfound.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [/Users/ciaraspencer/.wdm/drivers/chromedriver/mac64/97.0.4692.71/chromedriver] found in cache


coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [14]:
geo_id_list =geo_id 

Length Check

In [15]:
print(len(geo_id_list))

10904


Matching FIPS to DataFrame

In [16]:
#for geocode file
fips_fill = fips_check_notna #confirm pair to notna DataFrame
#if FIPS available
#fips_fill = fips_check_scraped_null.iloc[:8655]

In [17]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

<ipython-input-17-08e9c917fe2e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,full_add,Lat,Long,FIPS_z
0,1517597200,"204 20TH ST N, BIRMINGHAM, AL",33.515571,-86.806634,[010730027012041]
1,1798427201,"2090 Columbiana Road, VESTAVIA HILLS, AL",33.435789,-86.811428,[010730144043001]
2,2886767410,"3700 Cahaba Beach Road, BIRMINGHAM, AL",33.431916,-86.689670,[011170302122003]
3,4982277110,"1901 6TH AVE N Ste 2400, BIRMINGHAM, AL",33.518619,-86.809388,[010730027025027]
4,5001327110,"VESTAVIA PKWY, VESTAVIA, AL",33.418736,-86.797244,[010730129161002]
...,...,...,...,...,...
10907,9206768503,"1575 Old Fort Rd E, Fort Deposit, AL",31.975720,-86.603696,[010857812001055]
10908,9298908504,"685 Fox Trl, Opelika, AL",32.650615,-85.358977,[010810416003005]
10909,9323947403,"221 Wagon Trail, Alabaster, AL",33.256737,-86.828368,[011170303201008]
10910,9406888509,"5685B I 10 Industrial Pkwy N, Theodore, AL",30.568623,-88.175929,[010970071012016]


Remove brackets

In [18]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

<ipython-input-18-e19831f223ac>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,full_add,Lat,Long,FIPS_z
0,1517597200,"204 20TH ST N, BIRMINGHAM, AL",33.515571,-86.806634,010730027012041
1,1798427201,"2090 Columbiana Road, VESTAVIA HILLS, AL",33.435789,-86.811428,010730144043001
2,2886767410,"3700 Cahaba Beach Road, BIRMINGHAM, AL",33.431916,-86.689670,011170302122003
3,4982277110,"1901 6TH AVE N Ste 2400, BIRMINGHAM, AL",33.518619,-86.809388,010730027025027
4,5001327110,"VESTAVIA PKWY, VESTAVIA, AL",33.418736,-86.797244,010730129161002
...,...,...,...,...,...
10907,9206768503,"1575 Old Fort Rd E, Fort Deposit, AL",31.975720,-86.603696,010857812001055
10908,9298908504,"685 Fox Trl, Opelika, AL",32.650615,-85.358977,010810416003005
10909,9323947403,"221 Wagon Trail, Alabaster, AL",33.256737,-86.828368,011170303201008
10910,9406888509,"5685B I 10 Industrial Pkwy N, Theodore, AL",30.568623,-88.175929,010970071012016


In [19]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check

,LoanNumber,full_add,Lat,Long,FIPS_z
212,6205747005,"1980 hwy 315, SAGINAW, AL",34.212860,-89.696248,NaN
572,5260357102,"5015 Business Park BLVD, NEW MARKET, AL",61.174949,-149.888309,NaN
1277,8999377001,"508 GREENE STREET, GREENSBORO, AL",36.078119,-79.790771,NaN
1507,9686247001,"11180 Highway 82, MC SHAN, AL",33.352678,-93.396758,NaN
1831,1300428408,"10350 Highway 80 E, Montgomery, AL",32.715967,-96.120698,NaN
2586,9072157102,"450 STATE LINE RD, WILMER, AL",30.675558,-88.412497,NaN
3141,6703258010,"4100 GEORGE J. BEAN PKWY SPACE 1-2-2687, MOBIL...",27.977152,-82.531138,NaN
3144,8161698300,"509, MADISON, AL",40.759209,-73.974410,NaN
3272,7149097206,"11070 Hwy 80 East, MOUNT MEIGS, AL",32.358810,-89.549259,NaN
4870,6549927102,"450 STATE LINE RD, WILMER, AL",30.675558,-88.412497,NaN


Parse for Merge

In [20]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
0,1517597200,010730027012041
1,1798427201,010730144043001
2,2886767410,011170302122003
3,4982277110,010730027025027
4,5001327110,010730129161002
...,...,...
10907,9206768503,010857812001055
10908,9298908504,010810416003005
10909,9323947403,011170303201008
10910,9406888509,010970071012016


Read in base data

In [27]:
#if from geocode file
fips_check = pd.read_csv("data/state_data/geo/geo_al.csv")
fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis =1) 
fips_check.head() #2663
'''
#if from FIPS file
fips_check = pd.read_csv("data/state_data/geo/geo_fips/AL_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head() #15796
'''

,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,...,Veteran,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long
0,2676,1517597200,4/15/20,459,PPP,ACTION ENTERPRISE HOLDINGS LLC,204 20TH ST N,BIRMINGHAM,AL,35203-3610,...,Non-Veteran,NaN,10124657.53,7/27/21,48.0,Transportation and Warehousing,ALABAMA DISTRICT OFFICE,"204 20TH ST N, BIRMINGHAM, AL",33.515571,-86.806634
1,2677,1798427201,4/15/20,459,PPP,NAPHCARE US INC,2090 Columbiana Road,VESTAVIA HILLS,AL,35216-2153,...,Veteran,NaN,10112876.71,6/11/21,62.0,Health Care and Social Assistance,ALABAMA DISTRICT OFFICE,"2090 Columbiana Road, VESTAVIA HILLS, AL",33.435789,-86.811428
2,2678,2886767410,5/6/20,459,PPP,"AMERICAN FAMILY CARE, INC.",3700 Cahaba Beach Road,BIRMINGHAM,AL,35242,...,Unanswered,NaN,10108611.11,6/11/21,62.0,Health Care and Social Assistance,ALABAMA DISTRICT OFFICE,"3700 Cahaba Beach Road, BIRMINGHAM, AL",33.431916,-86.689670
3,2679,4982277110,4/13/20,459,PPP,"MAYNARD COOPER & GALE, PC.",1901 6TH AVE N Ste 2400,BIRMINGHAM,AL,35203-2602,...,Unanswered,NaN,10132328.77,8/18/21,54.0,"Professional, Scientific, and Technical Services",ALABAMA DISTRICT OFFICE,"1901 6TH AVE N Ste 2400, BIRMINGHAM, AL",33.518619,-86.809388
4,2680,5001327110,4/13/20,459,PPP,"QUALITY RESTAURANT CONCEPTS, LLC",VESTAVIA PKWY,VESTAVIA,AL,35216-1327,...,Non-Veteran,NaN,10114722.22,6/11/21,72.0,Accomodation and Food Services,ALABAMA DISTRICT OFFICE,"VESTAVIA PKWY, VESTAVIA, AL",33.418736,-86.797244


Merge with MainFrame

In [28]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,...,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z
0,2676,1517597200,4/15/20,459,PPP,ACTION ENTERPRISE HOLDINGS LLC,204 20TH ST N,BIRMINGHAM,AL,35203-3610,...,NaN,10124657.53,7/27/21,48.0,Transportation and Warehousing,ALABAMA DISTRICT OFFICE,"204 20TH ST N, BIRMINGHAM, AL",33.515571,-86.806634,010730027012041
1,2677,1798427201,4/15/20,459,PPP,NAPHCARE US INC,2090 Columbiana Road,VESTAVIA HILLS,AL,35216-2153,...,NaN,10112876.71,6/11/21,62.0,Health Care and Social Assistance,ALABAMA DISTRICT OFFICE,"2090 Columbiana Road, VESTAVIA HILLS, AL",33.435789,-86.811428,010730144043001
2,2678,2886767410,5/6/20,459,PPP,"AMERICAN FAMILY CARE, INC.",3700 Cahaba Beach Road,BIRMINGHAM,AL,35242,...,NaN,10108611.11,6/11/21,62.0,Health Care and Social Assistance,ALABAMA DISTRICT OFFICE,"3700 Cahaba Beach Road, BIRMINGHAM, AL",33.431916,-86.689670,011170302122003
3,2679,4982277110,4/13/20,459,PPP,"MAYNARD COOPER & GALE, PC.",1901 6TH AVE N Ste 2400,BIRMINGHAM,AL,35203-2602,...,NaN,10132328.77,8/18/21,54.0,"Professional, Scientific, and Technical Services",ALABAMA DISTRICT OFFICE,"1901 6TH AVE N Ste 2400, BIRMINGHAM, AL",33.518619,-86.809388,010730027025027
4,2680,5001327110,4/13/20,459,PPP,"QUALITY RESTAURANT CONCEPTS, LLC",VESTAVIA PKWY,VESTAVIA,AL,35216-1327,...,NaN,10114722.22,6/11/21,72.0,Accomodation and Food Services,ALABAMA DISTRICT OFFICE,"VESTAVIA PKWY, VESTAVIA, AL",33.418736,-86.797244,010730129161002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10907,13583,9206768503,3/12/21,459,PPS,PRIESTER RETAIL STORES INC.,1575 Old Fort Rd E,Fort Deposit,AL,36032-4743,...,NaN,150604.11,8/11/21,42.0,Wholesale Trade,ALABAMA DISTRICT OFFICE,"1575 Old Fort Rd E, Fort Deposit, AL",31.975720,-86.603696,010857812001055
10908,13584,9298908504,3/12/21,459,PPS,AGS DISTRIBUTION INC.,685 Fox Trl,Opelika,AL,36801-4639,...,NaN,NaN,NaN,33.0,Manufacturing,ALABAMA DISTRICT OFFICE,"685 Fox Trl, Opelika, AL",32.650615,-85.358977,010810416003005
10909,13585,9323947403,5/20/20,459,PPP,ROBERTO REYES DRYWALL INC,221 Wagon Trail,Alabaster,AL,35007-5823,...,NaN,151841.10,8/13/21,23.0,Construction,ALABAMA DISTRICT OFFICE,"221 Wagon Trail, Alabaster, AL",33.256737,-86.828368,011170303201008
10910,13586,9406888509,3/12/21,459,PPS,OAKRIDGE SERVICES LLC,5685B I 10 Industrial Pkwy N,Theodore,AL,36582-1661,...,NaN,150427.40,8/27/21,56.0,Administrative and Support and Waste Managemen...,ALABAMA DISTRICT OFFICE,"5685B I 10 Industrial Pkwy N, Theodore, AL",30.568623,-88.175929,010970071012016


Fill FIPS Columns

In [ ]:
#fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
#fips_merge

Drop Extra FIPS Columns

In [ ]:
#fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
#fips_merge_drop

In [33]:
fips_merge.loc[fips_merge["FIPS_z"].isnull()] #verify 

,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,...,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z
212,2888,6205747005,4/6/20,459,PPP,SAGINAW PIPE,1980 hwy 315,SAGINAW,AL,35137,...,NaN,NaN,NaN,33.0,Manufacturing,ALABAMA DISTRICT OFFICE,"1980 hwy 315, SAGINAW, AL",34.212860,-89.696248,NaN
572,3248,5260357102,4/13/20,459,PPP,"YULISTA INTEGRATED SOLUTIONS, LLC",5015 Business Park BLVD,NEW MARKET,AL,35761,...,NaN,NaN,NaN,33.0,Manufacturing,ALABAMA DISTRICT OFFICE,"5015 Business Park BLVD, NEW MARKET, AL",61.174949,-149.888309,NaN
1097,3773,6283588301,1/26/21,459,PPS,RANGER ENVIRONMENTAL SERVICES LLC,10601 Highway 43,Creola,AL,36525-4537,...,NaN,NaN,NaN,56.0,Administrative and Support and Waste Managemen...,ALABAMA DISTRICT OFFICE,"10601 Highway 43, Creola, AL",NaN,NaN,NaN
1277,3953,8999377001,4/9/20,459,PPP,HALE COUNTY HEALTH CARE AUTHORITY,508 GREENE STREET,GREENSBORO,AL,36744-2316,...,Y,934162.03,1/25/21,62.0,Health Care and Social Assistance,ALABAMA DISTRICT OFFICE,"508 GREENE STREET, GREENSBORO, AL",36.078119,-79.790771,NaN
1507,4183,9686247001,4/9/20,459,PPP,"MCSHAN LUMBER COMPANY, INC.",11180 Highway 82,MC SHAN,AL,35471-0027,...,NaN,826100.23,11/3/20,32.0,Manufacturing,ALABAMA DISTRICT OFFICE,"11180 Highway 82, MC SHAN, AL",33.352678,-93.396758,NaN
1831,4507,1300428408,2/1/21,459,PPS,BUCKMASTERS LTD.,10350 Highway 80 E,Montgomery,AL,36117-6064,...,NaN,NaN,NaN,42.0,Wholesale Trade,ALABAMA DISTRICT OFFICE,"10350 Highway 80 E, Montgomery, AL",32.715967,-96.120698,NaN
2586,5262,9072157102,4/15/20,459,PPP,"JAMES R. FINCHER TIMBER COMPANY, INC.",450 STATE LINE RD,WILMER,AL,36587-2001,...,NaN,553181.67,9/9/21,11.0,"Agriculture, Forestry, Fishing and Hunting",ALABAMA DISTRICT OFFICE,"450 STATE LINE RD, WILMER, AL",30.675558,-88.412497,NaN
3141,5817,6703258010,6/30/20,459,PPP,"HBF TAMPA PARTNERS JV, LLC",4100 GEORGE J. BEAN PKWY SPACE 1-2-2687,MOBILE,AL,36607,...,NaN,303687.71,4/6/21,72.0,Accomodation and Food Services,ALABAMA DISTRICT OFFICE,"4100 GEORGE J. BEAN PKWY SPACE 1-2-2687, MOBIL...",27.977152,-82.531138,NaN
3144,5820,8161698300,1/29/21,459,PPS,RUGGED PORTABLE COMPUTERS LLC,509,MADISON,AL,35758,...,NaN,NaN,NaN,33.0,Manufacturing,ALABAMA DISTRICT OFFICE,"509, MADISON, AL",40.759209,-73.974410,NaN
3272,5948,7149097206,4/28/20,459,PPP,"DUBOSE CONSTRUCTION COMPANY, LLC",11070 Hwy 80 East,MOUNT MEIGS,AL,36057-0117,...,NaN,453011.07,1/13/21,23.0,Construction,ALABAMA DISTRICT OFFICE,"11070 Hwy 80 East, MOUNT MEIGS, AL",32.358810,-89.549259,NaN


In [35]:
fips_AL = fips_merge

Convert to File

In [48]:
geo_AL_fips.to_csv("data/state_data/geo/geo_fips/AL_fips_scraped.csv") 

Review & Compare

In [2]:
geo_AL_fips = pd.read_csv("data/state_data/geo/geo_fips/AL_fips_scraped.csv") 
geo_AL_fips

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,...,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z
0,0,0,0,0,2676,1517597200,4/15/20,459,PPP,ACTION ENTERPRISE HOLDINGS LLC,...,N,10124657.53,7/27/21,48.0,Transportation and Warehousing,ALABAMA DISTRICT OFFICE,"204 20TH ST N, BIRMINGHAM, AL",33.515571,-86.806634,1.073003e+13
1,1,1,1,1,2677,1798427201,4/15/20,459,PPP,NAPHCARE US INC,...,N,10112876.71,6/11/21,62.0,Health Care and Social Assistance,ALABAMA DISTRICT OFFICE,"2090 Columbiana Road, VESTAVIA HILLS, AL",33.435789,-86.811428,1.073014e+13
2,2,2,2,2,2678,2886767410,5/6/20,459,PPP,"AMERICAN FAMILY CARE, INC.",...,N,10108611.11,6/11/21,62.0,Health Care and Social Assistance,ALABAMA DISTRICT OFFICE,"3700 Cahaba Beach Road, BIRMINGHAM, AL",33.431916,-86.689670,1.117030e+13
3,3,3,3,3,2679,4982277110,4/13/20,459,PPP,"MAYNARD COOPER & GALE, PC.",...,N,10132328.77,8/18/21,54.0,"Professional, Scientific, and Technical Services",ALABAMA DISTRICT OFFICE,"1901 6TH AVE N Ste 2400, BIRMINGHAM, AL",33.518619,-86.809388,1.073003e+13
4,4,4,4,4,2680,5001327110,4/13/20,459,PPP,"QUALITY RESTAURANT CONCEPTS, LLC",...,N,10114722.22,6/11/21,72.0,Accomodation and Food Services,ALABAMA DISTRICT OFFICE,"VESTAVIA PKWY, VESTAVIA, AL",33.418736,-86.797244,1.073013e+13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10907,10907,10907,10907,10907,13583,9206768503,3/12/21,459,PPS,PRIESTER RETAIL STORES INC.,...,N,150604.11,8/11/21,42.0,Wholesale Trade,ALABAMA DISTRICT OFFICE,"1575 Old Fort Rd E, Fort Deposit, AL",31.975720,-86.603696,1.085781e+13
10908,10908,10908,10908,10908,13584,9298908504,3/12/21,459,PPS,AGS DISTRIBUTION INC.,...,N,NaN,NaN,33.0,Manufacturing,ALABAMA DISTRICT OFFICE,"685 Fox Trl, Opelika, AL",32.650615,-85.358977,1.081042e+13
10909,10909,10909,10909,10909,13585,9323947403,5/20/20,459,PPP,ROBERTO REYES DRYWALL INC,...,N,151841.10,8/13/21,23.0,Construction,ALABAMA DISTRICT OFFICE,"221 Wagon Trail, Alabaster, AL",33.256737,-86.828368,1.117030e+13
10910,10910,10910,10910,10910,13586,9406888509,3/12/21,459,PPS,OAKRIDGE SERVICES LLC,...,N,150427.40,8/27/21,56.0,Administrative and Support and Waste Managemen...,ALABAMA DISTRICT OFFICE,"5685B I 10 Industrial Pkwy N, Theodore, AL",30.568623,-88.175929,1.097007e+13


In [3]:
#fill in missing Lat, Long, FIPS
geo_AL_fips.loc[geo_AL_fips["FIPS_z"].isnull()]

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,...,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z
212,212,212,212,212,2888,6205747005,4/6/20,459,PPP,SAGINAW PIPE,...,N,NaN,NaN,33.0,Manufacturing,ALABAMA DISTRICT OFFICE,"1980 hwy 315, SAGINAW, AL",34.212860,-89.696248,NaN
572,572,572,572,572,3248,5260357102,4/13/20,459,PPP,"YULISTA INTEGRATED SOLUTIONS, LLC",...,N,NaN,NaN,33.0,Manufacturing,ALABAMA DISTRICT OFFICE,"5015 Business Park BLVD, NEW MARKET, AL",61.174949,-149.888309,NaN
1097,1097,1097,1097,1097,3773,6283588301,1/26/21,459,PPS,RANGER ENVIRONMENTAL SERVICES LLC,...,N,NaN,NaN,56.0,Administrative and Support and Waste Managemen...,ALABAMA DISTRICT OFFICE,"10601 Highway 43, Creola, AL",NaN,NaN,NaN
1277,1277,1277,1277,1277,3953,8999377001,4/9/20,459,PPP,HALE COUNTY HEALTH CARE AUTHORITY,...,Y,934162.03,1/25/21,62.0,Health Care and Social Assistance,ALABAMA DISTRICT OFFICE,"508 GREENE STREET, GREENSBORO, AL",36.078119,-79.790771,NaN
1507,1507,1507,1507,1507,4183,9686247001,4/9/20,459,PPP,"MCSHAN LUMBER COMPANY, INC.",...,N,826100.23,11/3/20,32.0,Manufacturing,ALABAMA DISTRICT OFFICE,"11180 Highway 82, MC SHAN, AL",33.352678,-93.396758,NaN
1831,1831,1831,1831,1831,4507,1300428408,2/1/21,459,PPS,BUCKMASTERS LTD.,...,N,NaN,NaN,42.0,Wholesale Trade,ALABAMA DISTRICT OFFICE,"10350 Highway 80 E, Montgomery, AL",32.715967,-96.120698,NaN
2586,2586,2586,2586,2586,5262,9072157102,4/15/20,459,PPP,"JAMES R. FINCHER TIMBER COMPANY, INC.",...,N,553181.67,9/9/21,11.0,"Agriculture, Forestry, Fishing and Hunting",ALABAMA DISTRICT OFFICE,"450 STATE LINE RD, WILMER, AL",30.675558,-88.412497,NaN
3141,3141,3141,3141,3141,5817,6703258010,6/30/20,459,PPP,"HBF TAMPA PARTNERS JV, LLC",...,N,303687.71,4/6/21,72.0,Accomodation and Food Services,ALABAMA DISTRICT OFFICE,"4100 GEORGE J. BEAN PKWY SPACE 1-2-2687, MOBIL...",27.977152,-82.531138,NaN
3144,3144,3144,3144,3144,5820,8161698300,1/29/21,459,PPS,RUGGED PORTABLE COMPUTERS LLC,...,N,NaN,NaN,33.0,Manufacturing,ALABAMA DISTRICT OFFICE,"509, MADISON, AL",40.759209,-73.974410,NaN
3272,3272,3272,3272,3272,5948,7149097206,4/28/20,459,PPP,"DUBOSE CONSTRUCTION COMPANY, LLC",...,N,453011.07,1/13/21,23.0,Construction,ALABAMA DISTRICT OFFICE,"11070 Hwy 80 East, MOUNT MEIGS, AL",32.358810,-89.549259,NaN
